# Testing

> Testing

- skip_showdoc: true
- skip_exec: true

## 3 levels

- direct testing (python api)
- DRF testing (direct json)
- server testing (full json api)


## Setting Up the Testing Environment

> Ensure you have `pytest` and `pytest-django` installed. These are popular packages that enhance Django's built-in testing framework.

```sh
pip install pytest pytest-django
```

### Configure pytest:


> Create a `pytest.ini` file in the root of your project to configure `pytest`:

```ini
[pytest]
DJANGO_SETTINGS_MODULE = myproject.settings
python_files = tests.py test_*.py *_tests.py

```

Or
> Add to `pyproject.toml`
```yaml
[tool.pytest.ini_options]
DJANGO_SETTINGS_MODULE = "rm.settings"
python_files = ["*test*.py"]
```

## Writing Tests

### Model Tests

> Model tests ensure that your models behave as expected.

```python
# questionnaire/tests/test_models.py
import pytest
from questionnaire.models import Questionnaire, Question, Choice, Response, Answer
from base.models import Client

@pytest.mark.django_db
def test_create_questionnaire():
    questionnaire = Questionnaire.objects.create(title="Test Questionnaire", description="A test questionnaire.")
    assert questionnaire.title == "Test Questionnaire"
    assert questionnaire.description == "A test questionnaire."

```

### Serializer Tests

```python
# questionnaire/tests/test_serializers.py
import pytest
from questionnaire.serializers import QuestionnaireSerializer
from questionnaire.models import Questionnaire

@pytest.mark.django_db
def test_questionnaire_serializer():
    data = {"title": "Test Questionnaire", "description": "A test questionnaire."}
    serializer = QuestionnaireSerializer(data=data)
    assert serializer.is_valid()
    questionnaire = serializer.save()
    assert questionnaire.title == "Test Questionnaire"


```

### Testing Views and Endpoints


```python
# questionnaire/tests/test_views.py
import pytest
from django.urls import reverse
from rest_framework import status
from questionnaire.models import Questionnaire

@pytest.mark.django_db
def test_create_questionnaire(api_client):
    url = reverse('questionnaire-list')
    data = {"title": "Test Questionnaire", "description": "A test questionnaire."}
    response = api_client.post(url, data, format='json')
    assert response.status_code == status.HTTP_201_CREATED
    assert Questionnaire.objects.count() == 1
    assert Questionnaire.objects.get().title == "Test Questionnaire"


```

## Testing with Fixtures

```python
# questionnaire/tests/test_views_with_fixtures.py
import pytest
from django.urls import reverse
from rest_framework import status
from base.models import Client
from questionnaire.models import Questionnaire, Question, Response

@pytest.fixture
def client_data():
    return {"name": "Test Client", "email": "testclient@example.com", "sex": "M", "dob": "1990-01-01"}

@pytest.fixture
def client(client_data):
    return Client.objects.create(**client_data)

@pytest.fixture
def questionnaire():
    return Questionnaire.objects.create(title="Test Questionnaire", description="A test questionnaire.")

@pytest.mark.django_db
def test_create_response(api_client, client, questionnaire):
    url = reverse('response-list')
    data = {"questionnaire": questionnaire.id, "client": client.id}
    response = api_client.post(url, data, format='json')
    assert response.status_code == status.HTTP_201_CREATED
    assert Response.objects.count() == 1
    assert Response.objects.get().client == client

```